# Perceptron
## Machine Learning

---

In [2]:
import numpy as np
import pandas as pd

We use 4 attributions (the first 4 columns)

1. variance of Wavelet Transformed image (continuous) 
2. skewness of Wavelet Transformed image (continuous) 
3. curtosis of Wavelet Transformed image (continuous) 
4. entropy of image (continuous) 

The label is the last column: genuine or forged

In [25]:
test_data = pd.read_csv('bank-note/test.csv', header=None)
train_data = pd.read_csv('bank-note/train.csv', header=None)

In [72]:
train_data.shape

(872, 5)

In [26]:
train_data.head()

,0,1,2,3,4
0,3.848100,10.15390,-3.85610,-4.22280,0
1,4.004700,0.45937,1.36210,1.61810,0
2,-0.048008,-1.60370,8.47560,0.75558,0
3,-1.266700,2.81830,-2.42600,-1.88620,1
4,2.203400,5.99470,0.53009,0.84998,0


In [27]:
# first 7 columns are features, last column (Slump) is output
columns = ['var', 'skew', 'curt', 'ent', 'label']
features = columns[:-1]
output = columns[-1]

test_data.columns = columns
train_data.columns = columns

In [28]:
train_data.head()

,var,skew,curt,ent,label
0,3.848100,10.15390,-3.85610,-4.22280,0
1,4.004700,0.45937,1.36210,1.61810,0
2,-0.048008,-1.60370,8.47560,0.75558,0
3,-1.266700,2.81830,-2.42600,-1.88620,1
4,2.203400,5.99470,0.53009,0.84998,0


In [89]:
train_inputs = train_data.iloc[:,:-1].values
test_inputs = test_data.iloc[:,:-1].values

In [90]:
train_labels = train_data.iloc[:,-1].values
test_labels = test_data.iloc[:,-1].values

---

### A) Standard Perceptron

In [282]:
class Perceptron(object):

    def __init__(self, no_of_inputs, epoch=10, rate=0.01):
        self.epoch = epoch
        self.rate = rate   # learning rate
        self.weights = np.zeros(no_of_inputs + 1)  # initialize weights to zero
           
    def predict(self, inputs):
        # predicts the label of one training example input with current weights
        summation = np.dot(inputs, self.weights[1:]) + self.weights[0]
        if summation > 0:
            activation = 1
        else:
            activation = 0            
        return activation

    def train(self, train_inputs, labels):
        # trains perceptron weights on training dataset
        labels = np.expand_dims(labels, axis=1)
        data = np.hstack((train_inputs,labels))
        for e in range(self.epoch):
            #print("Epoch: "+ str(e))
            #print("Weights: " + str(self.weights))
            np.random.shuffle(data)
            for row in data:
                inputs = row[:-1]
                label = row[-1]
                prediction = self.predict(inputs)
                self.weights[1:] += self.rate * (label - prediction) * inputs
                self.weights[0] += self.rate * (label - prediction)
                
        return self.weights
                
    def evaluate(self, test_inputs, labels):
        # calculates average prediction error on testing dataset
        errors = []
        for inputs, label in zip(test_inputs, labels):
            prediction = self.predict(inputs)
            errors.append(np.abs(label-prediction))
        
        return sum(errors) / float(test_inputs.shape[0])

In [291]:
perceptron = Perceptron(4)

In [292]:
perceptron.train(train_inputs, train_labels)

array([ 0.52      , -0.59570448, -0.3630103 , -0.41950593, -0.06221142])

In [293]:
perceptron.evaluate(train_inputs, train_labels)

0.011467889908256881

In [294]:
perceptron.evaluate(test_inputs, test_labels)

0.012

In [295]:
weights = []
errors = []

for i in range(100):
    perceptron = Perceptron(4)
    weights.append(perceptron.train(train_inputs, train_labels))
    errors.append(perceptron.evaluate(test_inputs, test_labels))
    
print(np.mean(weights, axis=0)), print(np.mean(errors))

[ 0.5609     -0.59135312 -0.38124022 -0.40615604 -0.09201845]
0.023100000000000006


(None, None)

---

### B) Voted Perceptron

In [392]:
class VotedPerceptron(object):

    def __init__(self, no_of_inputs, epoch=10, rate=0.01):
        self.epoch = epoch
        self.rate = rate   # learning rate
        self.weights = np.zeros(no_of_inputs + 1)  # initialize weights to zero
        #self.weights_set = [np.zeros(no_of_inputs + 1)]
        self.C = [0]
        
    def predict(self, inputs, weights):
        # predicts the label of one training example input with current weights
        summation = np.dot(inputs, weights[1:]) + weights[0]
        if summation > 0:
            activation = 1
        else:
            activation = 0            
        return activation

    def train(self, train_inputs, labels):
        # trains perceptron weights on training dataset
        weights = np.zeros(train_inputs.shape[1] + 1)
        weights_set = [np.zeros(train_inputs.shape[1]+1)]
        labels = np.expand_dims(labels, axis=1)
        data = np.hstack((train_inputs,labels))
        m = 0
        for e in range(self.epoch):
            #print("Epoch: "+ str(e))
            
            np.random.shuffle(data)
            for row in data:
                inputs = row[:-1]
                label = row[-1]
                prediction = self.predict(inputs, weights)
                error = label - prediction
                if error:
                    #weights_a = self.rate * (label - prediction) * inputs
                    #weights_b = self.rate * (label - prediction)
                    #self.weights[1:] += weights_a
                    #self.weights[0] += weights_b
                    weights[1:] += self.rate * (label - prediction) * inputs
                    weights[0] += self.rate * (label - prediction)
                    #print('Error!')
                    #print(weights)
                    weights_set.append(np.copy(weights))
                    
                    self.C.append(1)
                    m += 1
                    
                else:
                    self.C[m] += 1
                    
        self.weights = weights
        self.weights_set = weights_set
        
        return self.weights
                
    
    def evaluate(self, test_inputs, labels):
        # calculates average prediction error on testing dataset
        errors = []
        n_weights = len(self.weights_set)
        for inputs, label in zip(test_inputs, labels):
            predictions = []
            for k in range(n_weights):
                pred = self.predict(inputs, weights=self.weights_set[k])
                if not pred:
                    pred = -1
                predictions.append(self.C[k]*pred)
                
            prediction = np.sign(sum(predictions))
            if prediction == -1:
                prediction = 0
            
            errors.append(np.abs(label-prediction))
        
        return sum(errors) / float(test_inputs.shape[0])

In [397]:
perceptron = VotedPerceptron(4)
perceptron.train(train_inputs, train_labels)

array([ 0.54      , -0.57252862, -0.34178077, -0.44049389, -0.1817347 ])

In [398]:
perceptron.evaluate(train_inputs, train_labels)

0.011467889908256881

In [399]:
perceptron.evaluate(train_inputs, train_labels)
perceptron.evaluate(test_inputs, test_labels)

0.012

In [393]:
weights = []
errors = []

for i in range(100):
    perceptron = VotedPerceptron(4)
    weights.append(perceptron.train(train_inputs, train_labels))
    errors.append(perceptron.evaluate(test_inputs, test_labels))
    
print(np.mean(weights, axis=0)), print(np.mean(errors))

[ 0.5609     -0.59851385 -0.37898256 -0.41597761 -0.09158596]
0.013760000000000003


(None, None)

---

### C) Average Perceptron

In [407]:
class AvgPerceptron(object):

    def __init__(self, no_of_inputs, epoch=10, rate=0.01):
        self.epoch = epoch
        self.rate = rate   # learning rate
        self.weights = np.zeros(no_of_inputs + 1)  # initialize weights to zero
        #self.weights_set = [np.zeros(no_of_inputs + 1)]
        self.a = np.zeros(no_of_inputs + 1)
        
    def predict(self, inputs, weights):
        # predicts the label of one training example input with current weights
        summation = np.dot(inputs, weights[1:]) + weights[0]
        if summation > 0:
            activation = 1
        else:
            activation = 0            
        return activation

    def train(self, train_inputs, labels):
        # trains perceptron weights on training dataset
        weights = np.zeros(train_inputs.shape[1] + 1)
        weights_set = [np.zeros(train_inputs.shape[1]+1)]
        labels = np.expand_dims(labels, axis=1)
        data = np.hstack((train_inputs,labels))
        m = 0
        for e in range(self.epoch):
            #print("Epoch: "+ str(e))
            
            np.random.shuffle(data)
            for row in data:
                inputs = row[:-1]
                label = row[-1]
                prediction = self.predict(inputs, weights)
                error = label - prediction
                weights[1:] += self.rate * (label - prediction) * inputs
                weights[0] += self.rate * (label - prediction)
                self.a += np.copy(weights)
                    

        self.weights = weights
        
        return self.a
    
    def evaluate(self, test_inputs, labels):
        # calculates average prediction error on testing dataset
        errors = []
        for inputs, label in zip(test_inputs, labels):
            prediction = self.predict(inputs, weights=self.a)
            errors.append(np.abs(label-prediction))
        
        return sum(errors) / float(test_inputs.shape[0])

In [417]:
perceptron = AvgPerceptron(4)
perceptron.train(train_inputs, train_labels)

array([ 3358.96      , -3921.94546045, -2562.311736  , -2656.91071295,
        -884.42353634])

In [418]:
perceptron.evaluate(train_inputs, train_labels)

0.01261467889908257

In [419]:
perceptron.evaluate(test_inputs, test_labels)

0.014

In [420]:
weights = []
errors = []

for i in range(100):
    perceptron = AvgPerceptron(4)
    weights.append(perceptron.train(train_inputs, train_labels))
    errors.append(perceptron.evaluate(test_inputs, test_labels))
    
print(np.mean(weights, axis=0)), print(np.mean(errors))

[ 3437.8202     -3980.81967769 -2591.51978412 -2681.63009231
  -752.43529268]
0.013860000000000004


(None, None)

---

### D) Comparison